*This notebook will create a file with the following transformations/filters:*

- Filtered to include only the following columns: ('SR_NUMBER', 'SR_TYPE', 'OWNER_DEPARTMENT',
  'STATUS', 'CREATED_DATE', 'CLOSED_DATE', 'DUPLICATE', 'PARENT_SR_NUMBER', 'COMMUNITY_AREA',
  'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK', 'CREATED_MONTH')
- 311 Information-Only calls removed
- Legacy records removed
- Removed rows with no Ward or Community Area
- Added 'time_to_close_sec' column including time difference in seconds between record creation
  and closure

**Transformations/filters remaining:**

- Add 'number of children' column denoting how many duplicates a request has
- Filter out duplicates once first bullet point completed (this code is already in the notebook, we'll just need to move it and uncomment it)
- Create dummy columns for request type, department, community area, ward, and hour/day/month of request creation
- Think about what our 'time-to-close' threshold should be and apply that filter -- should we filter out all requests closed in under X minutes? Or should we filter out a request type entirely if more than X% of its requests are filled in under X minutes?

**Notes/resources**:

- pandas to_pickle function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_pickle.html
- pandas read_pickle function: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_pickle.html#pandas.read_pickle
- I downloaded the Chicago 311 CSV file locally from the data portal: https://data.cityofchicago.org/Service-Requests/311-Service-Requests/v6vf-nfxy; it's ~1.4 GBs. I have it in a folder called 'raw_data' in my local repo (I didn't push it because the file is too big). You'll need to do the same on your end (download the file and put it in a 'raw_data' folder in your local repo). Let me know if you have any trouble with this (e.g., you don't have space to save a 1.4 GB file or it takes too long to download)
- I don't think we should push another pickle file to GitHub until the final version, but open to what you think. I think we can add the final filters/transformations to this notebook.

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import datetime as dt

Read in downloaded CSV file:

In [3]:
chi_311 = pd.read_csv('../raw_data/chicago_311_requests.csv')

/Users/michaelfeldman/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11,12,18,19,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
chi_311.shape

(4080622, 37)

Filter dataframe to exclude info-only calls:

In [5]:
chi_311_filtered = chi_311[chi_311['SR_TYPE'] != '311 INFORMATION ONLY CALL']

In [6]:
chi_311_filtered.shape

(2702596, 37)

Filter for columns needed:

In [7]:
chi_311_filtered.columns

Index(['SR_NUMBER', 'SR_TYPE', 'SR_SHORT_CODE', 'OWNER_DEPARTMENT', 'STATUS',
       'CREATED_DATE', 'LAST_MODIFIED_DATE', 'CLOSED_DATE', 'STREET_ADDRESS',
       'CITY', 'STATE', 'ZIP_CODE', 'STREET_NUMBER', 'STREET_DIRECTION',
       'STREET_NAME', 'STREET_TYPE', 'DUPLICATE', 'LEGACY_RECORD',
       'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER', 'COMMUNITY_AREA', 'WARD',
       'ELECTRICAL_DISTRICT', 'ELECTRICITY_GRID', 'POLICE_SECTOR',
       'POLICE_DISTRICT', 'POLICE_BEAT', 'PRECINCT',
       'SANITATION_DIVISION_DAYS', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
       'CREATED_MONTH', 'X_COORDINATE', 'Y_COORDINATE', 'LATITUDE',
       'LONGITUDE', 'LOCATION'],
      dtype='object')

In [8]:
chi_311_filtered = chi_311_filtered[['SR_NUMBER', 'SR_TYPE', 'OWNER_DEPARTMENT',
                                     'STATUS', 'CREATED_DATE', 'CLOSED_DATE', 'DUPLICATE',
                                     'LEGACY_RECORD', 'LEGACY_SR_NUMBER', 'PARENT_SR_NUMBER',
                                     'COMMUNITY_AREA', 'WARD', 'CREATED_HOUR', 'CREATED_DAY_OF_WEEK',
                                     'CREATED_MONTH']]

In [9]:
chi_311_filtered.shape

(2702596, 15)

Filter out legacy records:

In [10]:
legacy = chi_311_filtered[chi_311_filtered['LEGACY_RECORD'] == True]

In [11]:
max(legacy['CREATED_DATE'])

'12/18/2018 12:47:28 AM'

In [12]:
legacy.shape

(421791, 15)

In [13]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['LEGACY_RECORD'] == False]
chi_311_filtered = chi_311_filtered.drop(columns=['LEGACY_RECORD', 'LEGACY_SR_NUMBER'])

In [14]:
chi_311_filtered.shape

(2280805, 13)

Filter out rows with no community area or ward

In [15]:
chi_311_filtered[chi_311_filtered['COMMUNITY_AREA'].isna()].shape

(26835, 13)

In [16]:
chi_311_filtered[chi_311_filtered['WARD'].isna()].shape

(26655, 13)

In [17]:
chi_311_filtered = chi_311_filtered[chi_311_filtered['COMMUNITY_AREA'].notna() &
                                    chi_311_filtered['WARD'].notna()]

In [18]:
chi_311_filtered.shape

(2253959, 13)

(COMMENTED OUT THIS CODE) Check and remove duplicates:

**Do we want to add a column with the total number of children that a request has?**

In [19]:
dupes = chi_311_filtered[chi_311_filtered['DUPLICATE'] == True]

In [20]:
dupes[dupes['PARENT_SR_NUMBER'] == 'SR19-02206459']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
412,SR19-02557557,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/23/2019 06:13:16 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,18,2,9
674210,SR19-02412950,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/07/2019 06:59:08 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,6,7,9
766337,SR19-02786257,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,10/22/2019 02:16:57 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,14,3,10
2266793,SR19-01819033,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,06/28/2019 01:58:34 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,13,6,6
2266847,SR19-02412742,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/07/2019 05:11:59 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,5,7,9
2266848,SR19-02438203,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/10/2019 10:07:29 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,10,3,9
2266849,SR19-02438308,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/10/2019 10:11:17 AM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,10,3,9
2266867,SR19-02469775,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,09/12/2019 05:43:36 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,17,5,9
2266953,SR19-03092189,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,12/06/2019 10:44:39 PM,12/20/2019 03:19:51 PM,True,SR19-02206459,25.0,29.0,22,6,12


In [21]:
dupes[dupes['SR_NUMBER'] == 'SR19-02206459']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH


In [22]:
# Looks if we remove rows where duplicate == 'True', then we don't delete the original parent request
chi_311_filtered[chi_311_filtered['SR_NUMBER'] == 'SR19-02206459']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
761114,SR19-02206459,Pothole in Street Complaint,CDOT - Department of Transportation,Completed,08/14/2019 03:40:48 PM,12/20/2019 03:19:51 PM,False,NaN,25.0,29.0,15,4,8


In [23]:
dupes[dupes['PARENT_SR_NUMBER'] == 'SR19-01473136']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
40,SR19-01473141,Traffic Signal Out Complaint,CDOT - Department of Transportation,Open,05/11/2019 04:27:11 PM,NaN,True,SR19-01473136,55.0,10.0,16,7,5


In [24]:
chi_311_filtered[chi_311_filtered['SR_NUMBER'] == 'SR19-01473136']

,SR_NUMBER,SR_TYPE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,DUPLICATE,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
3809668,SR19-01473136,Traffic Signal Out Complaint,CDOT - Department of Transportation,Open,05/11/2019 04:26:20 PM,NaN,False,NaN,55.0,10.0,16,7,5


In [25]:
# chi_311_filtered = chi_311_filtered[chi_311_filtered['DUPLICATE'] == False]
# chi_311_filtered = chi_311_filtered.drop(columns=['DUPLICATE', 'PARENT_SR_NUMBER'])

In [26]:
# chi_311_filtered.shape

Filtered out complaints resolved in very short period of time -
NEED TO DECIDE WHICH COMPLAINTS TO FILTER OUT!!; do we want to filter out request types with more than a certain % fulfilled in less than X minutes?

In [27]:
chi_311_filtered['CREATED_DATE'] = pd.to_datetime(chi_311_filtered['CREATED_DATE'],
                                                  format='%m/%d/%Y %I:%M:%S %p')
chi_311_filtered['CLOSED_DATE'] = pd.to_datetime(chi_311_filtered['CLOSED_DATE'],
                                                format='%m/%d/%Y %I:%M:%S %p')

In [28]:
chi_311_filtered['time_to_close'] = chi_311_filtered['CLOSED_DATE'] - \
                                        chi_311_filtered['CREATED_DATE']

In [29]:
chi_311_filtered['time_to_close_sec'] = chi_311_filtered['time_to_close'].dt.total_seconds()

In [46]:
chi_311_filtered = chi_311_filtered.drop(columns=['time_to_close'])

In [30]:
# Requests resolved in 0 seconds
chi_311_filtered[chi_311_filtered['time_to_close_sec'] == 0].shape

(402913, 15)

In [31]:
# Requests resolved in less than 1 minute
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 60].shape

(770925, 15)

In [32]:
# Requests resolved in less than 10 minute
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 600].shape

(846269, 15)

In [33]:
# Requests resolved in less than 1 day
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 86400].shape

(1097323, 15)

In [34]:
# Types of requests resolved in less than 10 minutes
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 600]['SR_TYPE'].value_counts()

Aircraft Noise Complaint                     584578
Weed Removal Request                          96013
Graffiti Removal Request                      81066
Sign Repair Request - All Other Signs         40096
Pothole in Street Complaint                    8264
                                              ...  
Home Buyer Program Info Request                   1
Clean and Green Program Request                   1
Protected Bike Lane - Debris Removal              1
Pavement Cave-In Inspection Request               1
Bungalow/Vintage Home Information Request         1
Name: SR_TYPE, Length: 78, dtype: int64

In [35]:
# Filter out requests resolved in less than a minute
# chi_311_filtered = chi_311_filtered[chi_311_filtered['time_to_close_sec'] >= 60]

In [36]:
chi_311_filtered[chi_311_filtered['time_to_close_sec'] < 600]['SR_TYPE'].value_counts()

Aircraft Noise Complaint                     584578
Weed Removal Request                          96013
Graffiti Removal Request                      81066
Sign Repair Request - All Other Signs         40096
Pothole in Street Complaint                    8264
                                              ...  
Home Buyer Program Info Request                   1
Clean and Green Program Request                   1
Protected Bike Lane - Debris Removal              1
Pavement Cave-In Inspection Request               1
Bungalow/Vintage Home Information Request         1
Name: SR_TYPE, Length: 78, dtype: int64

Pickle filtered file:

In [47]:
# chi_311_filtered.to_pickle("../pickle_files/chi_311.pkl")